# Data Loading and Preparation

Please uncomment (remove #) and run the following cell to get the necessary module to run the entire code 

In [ ]:
# !pip install numpy
# !pip install pandas
# !pip install matplotlib
# !pip install seaborn
# !pip install plotly
# !pip install scipy
# !pip install scikit-learn

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import scipy.stats as stats

In [ ]:
df = pd.read_csv('green_tripdata_2015-09.csv')

df.head()

In [ ]:
df.columns

In [ ]:
# This column's name has a space at the end of it, so changing the name

df.rename({'Trip_type ':'Trip_type'},axis=1,inplace=True)

In [ ]:
print('Number of rows: ', df.shape[0])
print('Number of columns: ', df.shape[1])

In [ ]:
df.info()

- Most of the columns are numeric in type
- E_hail_fee column has no not null value

In [ ]:
# Checking null values in different columns

df.isnull().sum()

Ehail_fee is a column that has no not null value, all 1494926 rows are null

In [ ]:
# dropping Ehail_fee column

df.drop('Ehail_fee', axis=1,inplace=True)

In [ ]:
# replacing Trip_type column's null value with a central tendancy value, using mode as this column seems a categorial one

r_null_trip = df[df['Trip_type'].isnull()]

null_replace_mode = df[df['VendorID'].isin(r_null_trip['VendorID']) 
   & df['Store_and_fwd_flag'].isin(r_null_trip['Store_and_fwd_flag']) 
   & df['Payment_type'].isin(r_null_trip['Payment_type'])]['Trip_type'].mode()[0]

df['Trip_type'].fillna(null_replace_mode, inplace=True)

In [ ]:
df.isna().sum()

No column has null value now

In [ ]:
# Fare amount should not be 0, so dropping those rows

df = df.drop(df[df['Fare_amount']==0].index)

In [ ]:
# Trip distance also should be greater than 0

df = df.drop(df[df['Trip_distance']==0].index)

In [ ]:
#Converting some columns that are seems to be categorical in nature 

df[['VendorID', 'RateCodeID', 'Payment_type', 'Trip_type']] = df[['VendorID', 'RateCodeID', 'Payment_type','Trip_type']].astype(str)

In [ ]:
# Outliers in different numeric columns 

for column in df.select_dtypes(include=['number']).columns:
        q1 = df[column].quantile(0.25)
        q3 = df[column].quantile(0.75)
        iqr = q3 - q1
        lower_range = q1 - 1.5 * iqr
        upper_range = q3 + 1.5 * iqr
        print('Outlier in ' +column+  ' ' , df[(df[column] < lower_range) | (df[column] > upper_range)].shape[0])

In [ ]:
# Droping Outliers

def outliersDetect(df):
    for column in df.select_dtypes(include=['number']).columns:
        q1 = df[column].quantile(0.25)
        q3 = df[column].quantile(0.75)
        iqr = q3 - q1
        lower_range = q1 - 1.5 * iqr
        upper_range = q3 + 1.5 * iqr
        outliers_in_column = df[(df[column] > lower_range) & (df[column] < upper_range)]
    
    return outliers_in_column

df_cleaned = outliersDetect(df)

In [ ]:
print('After cleaning Data')
print('Number of rows: ', df_cleaned.shape[0])
print('Number of columns: ', df_cleaned.shape[1])

In [ ]:
# Check for duplicates

df_cleaned[df_cleaned.duplicated()]

In [ ]:
# converting date type columns into datetime

df_cleaned['lpep_pickup_datetime']=pd.to_datetime(df_cleaned['lpep_pickup_datetime'])
df_cleaned['Lpep_dropoff_datetime']=pd.to_datetime(df_cleaned['Lpep_dropoff_datetime'])

# Trip Distance

In [ ]:
df_cleaned['Trip_distance'].describe()

In [ ]:
bins = [0, 3, 6, 9, 12, 15,18]

fig = px.histogram(df_cleaned 
                   , x='Trip_distance'
                   , title='Trip Distance Distribution'
                   , text_auto=True
                   )

fig.update_traces(xbins=dict(start=0, end=18, size=3))
fig.update_layout(bargap=0.2, title=dict(x=0.5,xanchor='center')
                  , xaxis_title = 'Trip Distance'
                  , xaxis=dict(tickvals=bins, range=[0, 18]) 
                  , yaxis_title = 'Number of Trips' 
                  )

fig.show()

It seems that less distance trip (0-3 miles) had been made much higher than any other trip distance. Long distance (over 10 miles) trip had low in number. 

Hypothesis test:
    Under 95% confidence level, number of trips completed had distance less than 5

    Null Hypothsis:
        Trips completed had distance level more or equal to 5
    Alternate Hypothesis:
        Trips completed had distance less than 5

In [ ]:
# Calculating z value

alpha = round((1- 0.95),2)

sample_mean = df_cleaned['Trip_distance'].mean()
hypo_mean = 5
standard_error = np.std(df_cleaned['Trip_distance'],ddof=1)

z_value = (sample_mean - hypo_mean) / standard_error
# z_value

In [ ]:
z_critical_value = stats.norm.ppf(alpha)
# z_critical_value

In [ ]:
if z_value <  z_critical_value:
    print("Reject the null hypothesis")
else:
  print("Fail to reject the null hypothesis")

So, under 95% confidence level number of trips made had not distance less than 5 

In [ ]:
# To work with hour and day segmenting date time in respective columns

df_cleaned['pickup_hour'] = df_cleaned['lpep_pickup_datetime'].dt.hour
df_cleaned['dropoff_hour'] = df_cleaned['Lpep_dropoff_datetime'].dt.hour
df_cleaned['week_day'] = df_cleaned['lpep_pickup_datetime'].dt.dayofweek

In [ ]:
pickup_hour_grouped_central = df_cleaned.groupby('pickup_hour')['Trip_distance'].agg(Total='count',Mean=np.mean,Median = np.median).reset_index()

In [ ]:
fig1 = px.bar(pickup_hour_grouped_central
              , x='pickup_hour'
              , y='Mean'
              , title='Mean Trip Distance by Pick up Hour of the Day'
              , text_auto=True
              )

fig1.update_layout(xaxis_title='Hour of the Day'
                   , xaxis=dict(dtick=1)
                   , yaxis_title='Mean Trip Distance'
                   , title=dict(x=0.5, xanchor='center')
                   )

fig1.show()

5th and 6th hour of the day has more average trip distance (over 3 miles). 5th hour has the highest

In [ ]:
fig2 = px.bar(pickup_hour_grouped_central
              , x='pickup_hour'
              , y='Median'
              , title='Median Trip Distance by Pick up Hour of the Day'
              , text_auto=True
              )

fig2.update_layout(xaxis_title='Hour of the Day'
                   , yaxis_title='Median Trip Distance'
                   , xaxis=dict(dtick=1)
                   , title=dict(x=0.5, xanchor='center')
                   )

fig2.show()

Similar to average value, median of trip distance was higher in 5th and 6th hour of the day, and 5th hour has the highest

In [ ]:
fig3 = px.line(pickup_hour_grouped_central
               , x='pickup_hour'
               , y=['Mean', 'Median']
               , title = 'Mean and Median of Trip Distance by Pick up Hour of the Day'
               , markers=True
)

fig3.update_layout(xaxis_title = 'Hour of the Day'
                   , xaxis=dict(dtick=1)
                   , yaxis_title = 'Trip Distance'
                   , title=dict(x=0.5, xanchor='center'))

fig3.show()

Mean and median trip distance follow the similar trend. Median is lesser than mean

In [ ]:
dropoff_hour_grouped_central = df_cleaned.groupby('dropoff_hour')['Trip_distance'].agg(Total='count',Mean=np.mean,Median = np.median).reset_index()

In [ ]:
fig4 = px.bar(dropoff_hour_grouped_central
              , x='dropoff_hour'
              , y='Mean'
              , title='Mean Trip Distance by Drop of Hour of the Day'
              , text_auto=True
              )

fig4.update_layout(xaxis_title='Hour of the Day'
                   , yaxis_title='Mean Trip Distance'
                   , xaxis=dict(dtick=1)
                   , title=dict(x=0.5, xanchor='center')
                   )

fig4.show()

In [ ]:
fig5 = px.line(dropoff_hour_grouped_central
               , x='dropoff_hour'
               , y=['Mean', 'Median']
               , title = 'Mean and Median of Trip Distance by Drop off Hour of the Day'
               , markers=True
)

fig5.update_layout(xaxis_title = 'Hour of the Day'
                   , yaxis_title = 'Trip Distance'
                   , xaxis=dict(dtick=1)
                   , title=dict(x=0.5, xanchor='center'))

fig5.show()

Similar to pickup hour drop off hour has highest average trip distance in 5th hour and its mean - median are in similar trend

In [ ]:
pick_drop_mean_median = pd.merge(pickup_hour_grouped_central, dropoff_hour_grouped_central,
                                 left_on='pickup_hour', right_on='dropoff_hour',
                                 suffixes=['_pickup','_dropoff']
                                 )

In [ ]:
fig6 = px.line(pick_drop_mean_median
               , x='pickup_hour'
               , y=['Mean_pickup', 'Mean_dropoff']
               , title='Mean Trip Distance by Pickup and Dropoff Hour'
               , markers=True)

fig6.update_layout(xaxis_title='Hour of the Day'
                   , xaxis=dict(dtick=1)
                   , yaxis_title='Mean Trip Distance'
                   , title=dict(x=0.5, xanchor='center')
)

fig6.show()

Mean trip distance by pickup and dropoff hour is quite identical

In [ ]:
pickup_day_grouped_central = df_cleaned.groupby(['week_day','pickup_hour'])['Trip_distance'].agg(Mean=np.mean).reset_index()

In [ ]:
fig7 = px.line(pickup_day_grouped_central
               , x='pickup_hour'
               , y='Mean'
               , title = 'Average Trip Distance by Hour in different Day'
               , color = 'week_day'
               , markers=True
)

fig7.update_layout(xaxis_title = 'Hour of the Day'
                   , yaxis_title = 'Trip Distance'
                   , xaxis=dict(dtick=1)
                   , title=dict(x=0.5, xanchor='center'))

fig7.show()

- Week day 6 has highest average trip distance in most of the hour (apart from 4th, 5th, 22nd hour)
- On week day 2 5th hour has highest average trip distance
- Most of the day see a fall of trip distance after 6th hour to at least around 11th hour 

In [ ]:
Vendor_pickup = df_cleaned.groupby(['VendorID','pickup_hour'])['Trip_distance'].agg(Mean=np.mean).reset_index()

In [ ]:
fig8 = px.line(Vendor_pickup
               , x='pickup_hour'
               , y='Mean'
               , title = 'Average Trip Distance in Hour by different Vendors'
               , color = 'VendorID'
               , markers=True
)

fig8.update_layout(xaxis_title = 'Hour of the Day'
                   , yaxis_title = 'Trip Distance'
                   , xaxis=dict(dtick=1)
                   , title=dict(x=0.5, xanchor='center'))

fig8.show()

Vendor 2 has higher average trip distance than Vendor 1 in most of the hour of the day

In [ ]:
Vendor_fare = df_cleaned.groupby(['VendorID','pickup_hour'])['Fare_amount'].agg(Mean=np.mean).reset_index()

In [ ]:
fig9 = px.line(Vendor_fare
               , x='pickup_hour'
               , y='Mean'
               , title = 'Average Fare in Hour by different Vendors'
               , color = 'VendorID'
               , markers=True)

fig9.update_layout(xaxis_title = 'Hour of the Day'
                   , xaxis=dict(dtick=1)
                   , yaxis_title = 'Average Fare'
                   , title=dict(x=0.5, xanchor='center'))

fig9.show()

Vendor 2 has higher average fare in different hour than Vendor 1

In [ ]:
Vendor_tip = df_cleaned.groupby(['VendorID','pickup_hour'])['Tip_amount'].agg(Mean=np.mean).reset_index()

In [ ]:
fig10 = px.line(Vendor_tip
               , x='pickup_hour'
               , y='Mean'
               , title='Average Tip in Hour by different Vendors'
               , color = 'VendorID'
               , markers=True
)

fig10.update_layout(xaxis_title = 'Hour of the Day'
                    , xaxis=dict(dtick=1)
                    , yaxis_title = 'Average Tip Amount'
                    , title=dict(x=0.5, xanchor='center'))

fig10.show()

Vendor 1 has higher average tip amount than Vendor 2

- Vendor 2 has higher trip distance in different hour so its average fare is higher
- Vendor 1 tends to receive more tip on average than Vendor 2 

# NYC area Airports Trips

3 airports in New York

- LaGuardia Airport, NY, USA (40.776863, -73.874069)
- John F. Kennedy International Airport, NY, USA (40.641766, -73.780968)
- Newark Liberty International Airport, NY, USA (40.689491, -74.174538)

Trips in (and near this co-ordiantes may be treated as NYC area airports)

In [ ]:
# For different pickup and drop off latitude - longitude which are close to airports (co ordinates are taken from google maps)

def Near_airport(row):
    if (
        ((row['Pickup_longitude'] > -73.785837 ) & (row['Pickup_longitude'] <-73.779513 ) &
       (row['Pickup_latitude'] > 40.634216) & (row['Pickup_latitude']<40.652198)) |  
        (
       (row['Dropoff_longitude'] > -73.785837) & (row['Dropoff_longitude'] < -73.779513) &
       (row['Dropoff_latitude'] > 40.634216) & (row['Dropoff_latitude']<40.652198)) ):
        
        return 'JFK'
    
    if (
        ((row['Pickup_longitude'] < -73.86041 ) & (row['Pickup_longitude'] > -73.888391) & 
       (row['Pickup_latitude'] < 40.766995) & (row['Pickup_latitude']>40.762737)) | 
        (
       (row['Dropoff_longitude'] < -73.86041) & (row['Dropoff_longitude'] > -73.888391) &
       (row['Dropoff_latitude'] < 40.766995) & (row['Dropoff_latitude']>40.762737)) ):
        
        return 'LAG'
    
    if (
        ((row['Pickup_longitude'] < -74.162437) & (row['Pickup_longitude'] > -74.192975) & # long and lat from google maps
       (row['Pickup_latitude'] < 40.709331) & (row['Pickup_latitude']>40.681820)) | 
        (
       (row['Dropoff_longitude'] < -74.162437) & (row['Dropoff_longitude'] > -74.192975) &
       (row['Dropoff_latitude'] < 40.709331) & (row['Dropoff_latitude']>40.681820)) ):
        
        return 'KEWR'
    else:
        return 'NoAir'

In [ ]:
df_cleaned['Near_Airport'] = df_cleaned.apply(Near_airport, axis=1) 

In [ ]:
df_cleaned['Near_Airport'].value_counts()

In [ ]:
print('Number of trips Near Airports:',df_cleaned[df_cleaned['Near_Airport']!='NoAir']['Fare_amount'].shape[0])

In [ ]:
df_cleaned[df_cleaned['Near_Airport'] != 'NoAir'][['Fare_amount', 'Trip_distance', 'Total_amount', 'Tip_amount']].agg('mean').reset_index().rename(columns={'index': 'Corresponding Column', 0: 'Mean'})

In [ ]:
df_cleaned[df_cleaned['Near_Airport'] == 'NoAir'][['Fare_amount', 'Trip_distance', 'Total_amount', 'Tip_amount']].agg('mean').reset_index().rename(columns={'index': 'Corresponding Column', 0: 'Mean'})


In [ ]:
week_airport_fare = df_cleaned[df_cleaned['Near_Airport'] != 'NoAir'].groupby(['week_day','Near_Airport'])['Fare_amount']\
.agg('mean').reset_index().rename(columns={'week_day': 'Week Day Number','Near_Airport': 'Airport', 'Fare_amount': 'Avg Fare'})

# week_airport_fare

In [ ]:
fig11 = px.line(week_airport_fare
              , x= 'Week Day Number'
              , y= 'Avg Fare'
              , title = 'Average Fare towards Airports by different day in Week'
              , color= 'Airport'
              , markers=True)

fig11.update_layout(xaxis_title = 'Day of week'
                   , yaxis_title = 'Average Fare'
                   , title=dict(x=0.5, xanchor='center'))

fig11.show()

- There are trip towards LaGuardia Airport and John F. Kennedy International Airport all the day of the week, but on 1st and 6th day of the week no trip towards Newark Liberty International Airport
- On average fare towards John F. Kennedy International Airport (JFK) is higher

In [ ]:
hour_airport_fare = df_cleaned[df_cleaned['Near_Airport'] != 'NoAir'].groupby(['pickup_hour','Near_Airport'])['Fare_amount']\
.agg('mean').reset_index().rename(columns={'pickup_hour': 'Pickup Hour','Near_Airport': 'Airport', 'Fare_amount': 'Avg Fare'})

# hour_airport_fare

In [ ]:
fig12 = px.line(hour_airport_fare
              , x= 'Pickup Hour'
              , y= 'Avg Fare'
              , title = 'Average Fare towards Airports by different Pickup Hour'
              , color= 'Airport'
              , markers=True)

fig12.update_layout(xaxis_title = 'Pickup Hour'
                    , xaxis=dict(dtick=1)
                    , yaxis_title = 'Average Fare'
                    , title=dict(x=0.5, xanchor='center'))

fig12.show()

Average fare towards John F. Kennedy International Airport in different pickup hour is higher than others

# Dealing Tip Percentage

In [ ]:
df_cleaned['Tip_percentage'] = (df_cleaned['Tip_amount']/df_cleaned['Total_amount'])*100

In [ ]:
wk_pickup_tip_percent = df_cleaned.groupby(['week_day','pickup_hour'])['Tip_percentage'].aggregate(Mean=np.mean).reset_index()

# wk_pickup_tip_percent

In [ ]:
fig13 = px.line(wk_pickup_tip_percent
                , x='pickup_hour'
                , y='Mean'
                , title='Average Tip Percentage by Hour on Different Day'
                , color='week_day'
                , markers=True
                , facet_col='week_day'
                , facet_col_wrap=7)

fig13.update_layout(yaxis_title='Tip Percentage'
                    , title=dict(x=0.5, xanchor='center')
                    , height=600)

fig13.update_xaxes(title_text='Pickup Hour', showticklabels=True)

fig13.show()

Irrespective to the average value most of the day follow a similar trend of average tip percentage in different hour

In [ ]:
wk_airport_tip_percent = df_cleaned[df_cleaned['Near_Airport'] != 'NoAir'].groupby(['week_day','Near_Airport'])['Tip_percentage'].aggregate(Mean=np.mean).reset_index()

# wk_airport_tip_percent

In [ ]:
fig14 = px.line(wk_airport_tip_percent
                , x='week_day'
                , y='Mean'
                , title='Average Tip Percentage towards Airports on Different Day'
                , color='Near_Airport'
                , markers=True
                , facet_col='Near_Airport'
                , facet_col_wrap=3
                )

fig14.update_layout(yaxis_title='Tip Percentage'
                    , title=dict(x=0.5, xanchor='center')
                    , height=600
                    )

fig14.update_xaxes(title_text='Weed Day', showticklabels=True)


fig14.show()

- 2nd day of the week sees the highest tip percentage in trip towards any airports (Newark Liberty International Airport)
- Other than 2nd day, John F. Kennedy International Airport has higher average tip percentage 

In [ ]:
wk_vendor_tip_percent = df_cleaned.groupby(['week_day','VendorID'])['Tip_percentage'].aggregate(Mean=np.mean).reset_index()

# wk_vendor_tip_percent

In [ ]:
fig15 = px.bar(wk_vendor_tip_percent
               , x='week_day'
               , y='Mean'
               , title='Average Tip Percentage of Different Vendors on Different Days'
               , color='VendorID'
               , facet_col='VendorID'
               , facet_col_wrap=2
               )

fig15.update_layout(yaxis_title='Tip Percentage'
                    , title=dict(x=0.5, xanchor='center')
                    , height=600
                    )

fig15.update_xaxes(title_text='Week Day', showticklabels=True)

fig15.show()

- Both vendors receive maximum average tip percentage of fare at the 5th day of week
- Vendor 1 has higher tip percentage in all the day of week than Vendor 2

## Predictive Model for Tip percentage

### Library and Data Preparation

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor  
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import mutual_info_classif, mutual_info_regression

In [ ]:
# These columns were not vital for model building

df_cleaned.drop(['dropoff_hour','Near_Airport','RateCodeID', 'Extra'], axis=True, inplace=True)

In [ ]:
# To deal with time and day I will use derived column pickup_hour and week_day, so removing these two columns

df_cleaned.drop(['lpep_pickup_datetime', 'Lpep_dropoff_datetime'], axis=True, inplace=True)

In [ ]:
# Returing back these columns to initial data type, as it will be needed in model fitting

df_cleaned[['VendorID', 'Payment_type']] = df_cleaned[['VendorID', 'Payment_type']].astype(int)
df_cleaned['Trip_type'] = df_cleaned['Trip_type'].astype(float)

In [ ]:
# Replacing N/Y in the column to make it a field of number value

df_cleaned['Store_and_fwd_flag'] = df_cleaned['Store_and_fwd_flag'].replace({'N':0,'Y':1})

### Target Variable and Spliting Train and Test Data

In [ ]:
y = df_cleaned['Tip_percentage']

In [ ]:
X = df_cleaned.drop('Tip_percentage',axis=1)

In [ ]:
# Calculating score for finding which feature variable is vital for target variable 

def mutl_scores(X, y):
    mutual_scores = mutual_info_regression(X, y)
    mutual_scores = pd.Series(mutual_scores, name="MI Scores", index=X.columns)
    mutual_scores = mutual_scores.sort_values(ascending=False)
    return mutual_scores

mi_scores = mutl_scores(X, y)

It takes a while to calculate the mutual relation score

In [ ]:
# mi_scores

In [ ]:
# Removing the feature variable that has very low impact (mi score), threshhold I use is my preference

def remove_unrelated(dfrm, mi_scores):
    return dfrm.loc[:, mi_scores > 0.005]

X=remove_unrelated(X, mi_scores)

In [ ]:
# spliting train and test data. 75% data is in the training data and 25% data in the test data

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.25, random_state = 29)

In [ ]:
X_train.shape

In [ ]:
X_test.shape

### Machine Learning Algorithm to Build Predictive Model

##### Linear Regression

In [ ]:
Model1 = LinearRegression().fit(X_train, y_train)

y_pred1=Model1.predict(X_test)

Model_mse1 = mean_squared_error(y_test, y_pred1)
Model_Rmse1 = sqrt(Model_mse1)

#### Decision Tree Regression

In [ ]:
Model2 = DecisionTreeRegressor(random_state = 29).fit(X_train, y_train)

y_pred2 = Model2.predict(X_test)

Model_mse2 = mean_squared_error(y_test, y_pred2)
Model_Rmse2 = sqrt(Model_mse2)

#### Random Forest Regression

In [ ]:
Model3 = RandomForestRegressor(n_estimators=10, random_state=29).fit(X_train, y_train)
y_pred3 = Model3.predict(X_test)

Model_mse3 = mean_squared_error(y_test, y_pred3)
Model_Rmse3 = sqrt(Model_mse3)

#### Gradient Boosting Regression

In [ ]:
Model4 = GradientBoostingRegressor(n_estimators=10, random_state=29).fit(X_train, y_train)
y_pred4 = Model4.predict(X_test)

Model_mse4 = mean_squared_error(y_test, y_pred4)
Model_Rmse4 = sqrt(Model_mse4)

##### Model Comparison

In [ ]:
compare_models = pd.DataFrame(
    {  'Model' : ['Linear','Decision Tree','Random Forest','Gradient Boosting'],
        'MSE'  : [Model_mse1, Model_mse2,Model_mse3,Model_mse4],
        'RMSE' : [Model_Rmse1,Model_Rmse2,Model_Rmse3,Model_Rmse4]
    })

compare_models

Among the model **Random Forest Regression** has lower mean square error and root mean square error value. So this model is suggested to make prediction for trip percentage.